In [1]:
import pickle

with open('/sise/home/etaylor/images/trichomes_patches/week5_3xzoom_regular_best_images/analysis/SAM_pred.pickle', 'rb') as f:
    SAM_pred = pickle.load(f)

with open('/sise/home/etaylor/images/trichomes_patches/week5_3xzoom_regular_best_images/analysis/ground_truth.pickle', 'rb') as f:
    ground_truth = pickle.load(f)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns

import sys
sys.path.append("..")  # Adds higher directory to python modules path.

from preprocess.image_preprocess import *


## Read Images

In [3]:
patches_path = "/sise/home/etaylor/images/trichomes_patches/week5_3xzoom_regular_best_images"
images_names = read_images_and_names(patches_path)

# SAM

In [4]:
using_colab = True

In [5]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    # !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    # !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth - already downloaded at first time

PyTorch version: 2.0.1+cu117
Torchvision version: 0.15.2+cu117
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-q0bdl53w
  Running command git clone --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-q0bdl53w
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
--2023-08-21 15:55:55--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.165.201.106, 18.165.201.34, 18.165.201.102, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.165.201.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth.1’

100%[====================================>] 2,564,550,879 40.0MB/s   in 74s    

2023-08-21 15:57:17

In [6]:
def show_anns(anns):
    # If there are no annotations, exit the function early
    if len(anns) == 0:
        return

    # Sort the annotations by area in descending order
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)

    # Get the current axis (plot) on which to draw
    ax = plt.gca()
    # Disable autoscaling of the axis
    ax.set_autoscale_on(False)

    # Create a transparent image of the same size as the first (largest) annotation
    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0

    # Loop over each annotation
    for ann in sorted_anns:
        # Get the mask for this annotation
        m = ann['segmentation']

        # Generate a random color for this mask (RGB + alpha, where alpha is 0.35)
        color_mask = np.concatenate([np.random.random(3), [0.35]])

        # Apply the color to the mask on the image
        img[m] = color_mask

    # Display the image with the colored masks
    ax.imshow(img)

In [7]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

Define the segmentation function that changes format to be equal to ground truth

In [8]:
def segments_patches_SAM(images_names, mask_generator):
    segmentation_dict = {}

    for image, name in images_names:
        mask = mask_generator.generate(image)
        num_segments = len(mask)

        # Initialize an array of False with the same shape as the segment masks
        instance_bitmap = np.zeros_like(mask[0]['segmentation'], dtype=bool)

        # create a single instance bitmap
        for seg in mask:
          instance_bitmap = np.logical_or(instance_bitmap, seg['segmentation'])

        segmentation_dict[name] = {
            'mask': mask,
            'num_segments': num_segments,
            "instance_bitmap": instance_bitmap,
        }

    return segmentation_dict


## Define measuring metrics

In [9]:
def calculate_precision_recall_per_segment(gt_dict, pred_dict, iou_threshold):
    TPs = 0 # True positives
    FPs = 0 # False positives
    FNs = 0 # False negatives
    iou_values = []  # list to store IOU values

    gt_segments = np.unique(gt_dict['instance_bitmap']) # List of unique segments in ground truth
    gt_segments = gt_segments[gt_segments != 0] # Exclude the background (0)

    matched_gt_segments = set() # Track which gt segments have been matched

    # Iterate over each predicted segment
    for segment in pred_dict['mask']:
        pred_mask = segment['segmentation']
        max_iou = 0
        best_match_gt_seg = None

        for seg in gt_segments:
            if seg not in matched_gt_segments:  # Only consider unmatched gt segments
                gt_mask = (gt_dict['instance_bitmap'] == seg) # mask for current gt segment
                intersection = np.logical_and(gt_mask, pred_mask)
                union = np.logical_or(gt_mask, pred_mask)
                iou = np.sum(intersection) / np.sum(union)

                if iou > max_iou:
                    max_iou = iou
                    best_match_gt_seg = seg

        if max_iou >= iou_threshold and best_match_gt_seg is not None:
            TPs += 1 # We have a true positive
            matched_gt_segments.add(best_match_gt_seg) # This gt segment has been matched
        else:
            FPs += 1 # We have a false positive

        iou_values.append(max_iou)  # Add the max IOU to the list

    # Calculate false negatives by subtracting the number of matched gt segments from total gt segments
    FNs = len(gt_segments) - len(matched_gt_segments)

    precision = TPs / (TPs + FPs) if TPs + FPs > 0 else 0
    recall = TPs / (TPs + FNs) if TPs + FNs > 0 else 0

    return precision, recall, (TPs, FPs, FNs), iou_values

def calculate_average_precision_recall(ground_truth, SAM_pred, iou_threshold):
    precision_values = []
    recall_values = []
    total_TPs = 0
    total_FPs = 0
    total_FNs = 0
    all_iou_values = []
    num_images = len(ground_truth)

    for name in ground_truth:
        precision, recall, (TPs, FPs, FNs), iou_values = calculate_precision_recall_per_segment(ground_truth[name], SAM_pred[name], iou_threshold)
        precision_values.append(precision)
        recall_values.append(recall)
        total_TPs += TPs
        total_FPs += FPs
        total_FNs += FNs
        all_iou_values.extend(iou_values)

    average_precision = np.mean(precision_values)
    average_recall = np.mean(recall_values)
    std_dev_precision = np.std(precision_values)
    std_dev_recall = np.std(recall_values)

    return average_precision, average_recall, std_dev_precision, std_dev_recall, (total_TPs, total_FPs, total_FNs), all_iou_values


def plot_confusion_matrix(TPs, FPs, FNs):
    confusion_matrix = [[TPs, FPs],
                        [FNs, 0]] # Note that we don't have TNs in this case

    # Create a heatmap
    plt.figure(figsize=(6,4))
    heatmap = sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')

    # Set labels
    heatmap.xaxis.set_ticklabels(['Positive', 'Negative'])
    heatmap.yaxis.set_ticklabels(['Positive', 'Negative'])
    heatmap.xaxis.tick_top() # x axis on top
    heatmap.xaxis.set_label_position('top')

    # Add axis labels
    plt.xlabel('Actual')
    plt.ylabel('Predicted')

    plt.show()

# Experiment to find best hyper params

In [10]:
from sklearn.model_selection import ParameterGrid
import pandas as pd

# Define the hyperparameters grid
param_grid = {
    'points_per_side': [32, 64],
    'pred_iou_thresh': [0.86, 0.88, 0.9],
    'stability_score_thresh': [0.92, 0.94, 0.96],
    'stability_score_offset': [1.0, 1.2],
    'box_nms_thresh': [0.7, 0.8],
    'crop_n_layers': [0, 1],
    'crop_nms_thresh': [0.7, 0.8],
    'crop_overlap_ratio': [512 / 1500, 2/3],
    'crop_n_points_downscale_factor': [1, 2],
    'min_mask_region_area': [0, 100]
}

# Generate all combinations of hyperparameters
grid = ParameterGrid(param_grid)

# Placeholder for the results
results = []

# Iterate over all combinations of hyperparameters
for params in grid:
    # Create a new SAM model with the current hyperparameters
    mask_generator = SamAutomaticMaskGenerator(model=sam, **params)

    # Predict masks for all images in the test set and calculate average precision and recall
    SAM_pred = segments_patches_SAM(images_names, mask_generator)

    # Calculate average precision and recall
    average_precision, average_recall, _, _, (total_TPs, total_FPs, total_FNs), _ = calculate_average_precision_recall(ground_truth, SAM_pred, iou_threshold=0.5)

    # Print the scores for this iteration
    print(f'Parameters: {params}')
    print(f'Average precision: {average_precision}')
    print(f'Average recall: {average_recall}\n')
    print(f'True Positives: {total_TPs}, False Positives: {total_FPs}, False Negatives: {total_FNs}\n')

    # Store the results
    result = params.copy()
    result['average_precision'] = average_precision
    result['average_recall'] = average_recall
    result['TP'] = total_TPs
    result['FP'] = total_FPs
    result['FN'] = total_FNs
    results.append(result)

# Convert the results to a DataFrame
df_results = pd.DataFrame(results)

# Sort the results by average precision and recall
df_results.sort_values(['average_precision', 'average_recall'], ascending=False, inplace=True)

# Save the results to a CSV file
df_results.to_csv('grid_search_results.csv', index=False)

# Print the best results
print(df_results.head())

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.92}
Average precision: 0.31848198435253083
Average recall: 0.942526942114811

True Positives: 1413, False Positives: 3037, False Negatives: 100

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.94}
Average precision: 0.34890904394459227
Average recall: 0.938001467868227

True Positives: 1405, False Positives: 2650, False Negatives: 108

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_o

In [1]:
scores = """
Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.92}
Average precision: 0.31848198435253083
Average recall: 0.942526942114811

True Positives: 1413, False Positives: 3037, False Negatives: 100

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.94}
Average precision: 0.34890904394459227
Average recall: 0.938001467868227

True Positives: 1405, False Positives: 2650, False Negatives: 108

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.96}
Average precision: 0.435488875698625
Average recall: 0.8957884106049799

True Positives: 1345, False Positives: 1789, False Negatives: 168

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.92}
Average precision: 0.33714011192547766
Average recall: 0.9398924245688723

True Positives: 1408, False Positives: 2793, False Negatives: 105

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.94}
Average precision: 0.37928942786528996
Average recall: 0.9265099334279095

True Positives: 1390, False Positives: 2312, False Negatives: 123

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.96}
Average precision: 0.4873456813417624
Average recall: 0.8257203568171022

True Positives: 1238, False Positives: 1345, False Negatives: 275

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.88, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.92}
Average precision: 0.3356211997612023
Average recall: 0.9409419747945494

True Positives: 1410, False Positives: 2811, False Negatives: 103

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.88, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.94}
Average precision: 0.3619092384338169
Average recall: 0.9375929711362009

True Positives: 1404, False Positives: 2499, False Negatives: 109

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.88, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.96}
Average precision: 0.4426849463830271
Average recall: 0.8957884106049799

True Positives: 1345, False Positives: 1733, False Negatives: 168

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.88, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.92}
Average precision: 0.3515990138992945
Average recall: 0.9394839278368461

True Positives: 1407, False Positives: 2616, False Negatives: 106

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.88, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.94}
Average precision: 0.3883368274283015
Average recall: 0.9261014366958833

True Positives: 1389, False Positives: 2218, False Negatives: 124

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.88, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.96}
Average precision: 0.4943437566174142
Average recall: 0.8257203568171022

True Positives: 1238, False Positives: 1306, False Negatives: 275

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.9, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.92}
Average precision: 0.36791117023280184
Average recall: 0.9352102840383489

True Positives: 1402, False Positives: 2429, False Negatives: 111

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.9, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.94}
Average precision: 0.389036663757853
Average recall: 0.9330039671844518

True Positives: 1398, False Positives: 2210, False Negatives: 115

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.9, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.96}
Average precision: 0.46009777877384894
Average recall: 0.8952117093362371

True Positives: 1344, False Positives: 1605, False Negatives: 169

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.9, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.92}
Average precision: 0.38066635482955824
Average recall: 0.9343649821786848

True Positives: 1400, False Positives: 2293, False Negatives: 113

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.9, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.94}
Average precision: 0.412173121674674
Average recall: 0.9249551320201671

True Positives: 1387, False Positives: 2002, False Negatives: 126

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 32, 'pred_iou_thresh': 0.9, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.96}
Average precision: 0.5100751458430782
Average recall: 0.8240370125732044

True Positives: 1236, False Positives: 1214, False Negatives: 277

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 64, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.92}
Average precision: 0.2706561111655015
Average recall: 0.9461545905741398

True Positives: 1418, False Positives: 3899, False Negatives: 95

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 64, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.94}
Average precision: 0.3001307723946212
Average recall: 0.9401610251597936

True Positives: 1409, False Positives: 3363, False Negatives: 104

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 64, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.96}
Average precision: 0.3695751402532942
Average recall: 0.9085978123903056

True Positives: 1366, False Positives: 2436, False Negatives: 147

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 64, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.92}
Average precision: 0.28806514926001425
Average recall: 0.9415220401540266

True Positives: 1411, False Positives: 3567, False Negatives: 102

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 64, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.94}
Average precision: 0.3281816280891604
Average recall: 0.9294467805314234

True Positives: 1395, False Positives: 2954, False Negatives: 118

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 64, 'pred_iou_thresh': 0.86, 'stability_score_offset': 1.2, 'stability_score_thresh': 0.96}
Average precision: 0.41578135388329795
Average recall: 0.8545162135574189

True Positives: 1286, False Positives: 1928, False Negatives: 227

Parameters: {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'crop_n_points_downscale_factor': 1, 'crop_nms_thresh': 0.7, 'crop_overlap_ratio': 0.3413333333333333, 'min_mask_region_area': 0, 'points_per_side': 64, 'pred_iou_thresh': 0.88, 'stability_score_offset': 1.0, 'stability_score_thresh': 0.92}
Average precision: 0.2881892807231586
Average recall: 0.9435612817766016

True Positives: 1414, False Positives: 3561, False Negatives: 99


"""

In [7]:
# Extracting data using regular expressions
parameters_pattern = re.compile(r"Parameters: ({.*?})")
avg_precision_pattern = re.compile(r"Average precision: (.*?)\n")
avg_recall_pattern = re.compile(r"Average recall: (.*?)\n")
true_positives_pattern = re.compile(r"True Positives: (\d+)")
false_positives_pattern = re.compile(r"False Positives: (\d+)")
false_negatives_pattern = re.compile(r"False Negatives: (\d+)")

parameters_matches = parameters_pattern.findall(scores)
avg_precision_matches = avg_precision_pattern.findall(scores)
avg_recall_matches = avg_recall_pattern.findall(scores)
true_positives_matches = true_positives_pattern.findall(scores)
false_positives_matches = false_positives_pattern.findall(scores)
false_negatives_matches = false_negatives_pattern.findall(scores)

# Creating a dataframe
df_regex = pd.DataFrame({
    'Parameters': [eval(match) for match in parameters_matches],
    'Average Precision': [float(match) for match in avg_precision_matches],
    'Average Recall': [float(match) for match in avg_recall_matches],
    'True Positives': [int(match) for match in true_positives_matches],
    'False Positives': [int(match) for match in false_positives_matches],
    'False Negatives': [int(match) for match in false_negatives_matches]
})


,Parameters,Average Precision,Average Recall,True Positives,False Positives,False Negatives
0,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.318482,0.942527,1413,3037,100
1,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.348909,0.938001,1405,2650,108
2,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.435489,0.895788,1345,1789,168
3,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.337140,0.939892,1408,2793,105
4,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.379289,0.926510,1390,2312,123
5,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.487346,0.825720,1238,1345,275
6,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.335621,0.940942,1410,2811,103
7,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.361909,0.937593,1404,2499,109
8,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.442685,0.895788,1345,1733,168
9,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.351599,0.939484,1407,2616,106


In [8]:
df_regex

,Parameters,Average Precision,Average Recall,True Positives,False Positives,False Negatives
0,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.318482,0.942527,1413,3037,100
1,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.348909,0.938001,1405,2650,108
2,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.435489,0.895788,1345,1789,168
3,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.337140,0.939892,1408,2793,105
4,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.379289,0.926510,1390,2312,123
5,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.487346,0.825720,1238,1345,275
6,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.335621,0.940942,1410,2811,103
7,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.361909,0.937593,1404,2499,109
8,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.442685,0.895788,1345,1733,168
9,"{'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...",0.351599,0.939484,1407,2616,106


In [10]:
# Sorting the dataframe based on "Average Precision" and "Average Recall" in descending order
sorted_df = df_regex.sort_values(by=['Average Precision', 'Average Recall'], ascending=False)

# Extracting the top row
best_parameters = sorted_df.iloc[0]

best_parameters

Parameters           {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...
Average Precision                                             0.510075
Average Recall                                                0.824037
True Positives                                                    1236
False Positives                                                   1214
False Negatives                                                    277
Name: 17, dtype: object

In [11]:
df = df_regex

# Finding the parameters with the highest precision
max_precision_row = df[df["Average Precision"] == df["Average Precision"].max()]

# Finding the parameters with the highest recall
max_recall_row = df[df["Average Recall"] == df["Average Recall"].max()]

max_precision_row, max_recall_row


(                                           Parameters  Average Precision  \
 17  {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...           0.510075   
 
     Average Recall  True Positives  False Positives  False Negatives  
 17        0.824037            1236             1214              277  ,
                                            Parameters  Average Precision  \
 18  {'box_nms_thresh': 0.7, 'crop_n_layers': 0, 'c...           0.270656   
 
     Average Recall  True Positives  False Positives  False Negatives  
 18        0.946155            1418             3899               95  )